In [ ]:
import torch
from torch import nn
import pandas as pd
import os
import numpy as np
import sklearn
import json

In [ ]:
from google.colab import drive
drive.mount('/content/MyDrive/')
data_path = './MyDrive/MyDrive/thesis/data/'

Drive already mounted at /content/MyDrive/; to attempt to forcibly remount, call drive.mount("/content/MyDrive/", force_remount=True).


# Data Loading

In [ ]:
#files = [ file for file in os.listdir(data_path)[:-1] if file.split('_')[3] == "10000.csv" ]
f = 'punch_norm_2000_10000.csv'
#f = 'punch_norm_raw_raw.csv'

#data
with open(os.path.join(data_path, f)) as fp:
  df = pd.read_csv(fp)


try:
  df = df.drop(['Unnamed: 0'], axis=1)
except:
  pass
print(df.head())

try:
  period_length = int( f.split('_')[2] )
  sampling_rate = int( f.split('_')[3].split('.')[0] )
  n_observations = period_length / (sampling_rate / 1000) + 1  #timestamp zero, therefore +1 * 3 axis

except:
  period_length = 'raw'
  sampling_rate = 'raw'

n_channels = 3
n_punches = max(df['id'])+1
max_length = max( (df[df['id']==idx].shape[0])*3 for idx in np.arange(n_punches))

print(period_length,", ", sampling_rate, "\n")

#Labels
with open(os.path.join(data_path, "labels.csv")) as fp:
  labels = pd.read_csv(fp)


labels.head()

          x         y         z  timestamp  id
0  6.793223 -2.383251  2.962314          0   0
1  6.317743 -3.380427  3.761405      10000   0
2  6.333654 -3.077143  3.530405      20000   0
3  5.765556 -3.123323  3.622862      30000   0
4  5.353406 -3.081780  3.578873      40000   0
2000 ,  10000 



,label
0,upper-cut
1,upper-cut
2,upper-cut
3,upper-cut
4,frontal


In [ ]:
assert df[df['id']==0].shape[0] == n_observations

# Data Pre-processing
1. Integer encoding of labels
2. Creating torch Dataset object + one-hot encoding
3. Apply torch DataLoader with shuffle

In [ ]:
#==============================================
#     1.
#==============================================

unpack_string = lambda df: df[:1].values[0]

def encode_label(label):
  if unpack_string(label)=='frontal':
    return 0
  if unpack_string(label)=='hook':
    return 1
  if unpack_string(label)=='upper-cut':
    return 2
  if unpack_string(label)=='no-action':
    return 3


encoded_labels = labels.apply(encode_label, axis=1)



In [ ]:
#==============================================
#     2.
#==============================================

from torch.utils.data import Dataset
from torch.nn.functional import one_hot
from sklearn.preprocessing import MinMaxScaler

class TwoDimPunchDataset(Dataset):
     def __init__(self, data, labels, train_indices, max_length):
         super(Dataset, self).__init__()

         self.data = data                               #to pandas DataFrame
         self.labels = torch.from_numpy(labels.values)  #to 1D tensor

         #Fit scaler on training data
         self.scaler = MinMaxScaler().fit( list( [self.__getitem__(idx, scale=False)[0] for idx in train_indices ] ) )

     def __len__(self):
        return len(self.labels)

     def __getitem__(self, index, scale=True):
        punch = self.data[self.data['id']==index]

        #concatenate all signals to form 1D numpy array
        signal = np.concatenate((punch['x'].values, punch['y'].values, punch['z'].values))
        dif = max_length - len(signal)
        #print(signal.shape)

        if dif != 0:
            signal = np.concatenate((signal, np.zeros(dif)))

        if scale==True:
            signal = self.scaler.transform(signal.reshape(1,-1))


        label = self.labels[index]
        label = one_hot(label, num_classes=4)           # One hot encoding of labels
        return signal, label

In [ ]:

from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler

batch_size = 16
random_state = 42
n_samples = 7606
indices = list(range(n_samples))
val_split = int(np.floor(0.1 * n_samples))
test_split = int(np.floor(0.2 * n_samples))


np.random.seed(random_state)
np.random.shuffle(indices)
val_indices, test_indices, train_indices =  indices[:val_split], indices[val_split:test_split], indices[test_split:]

#Takes a while to fit the scaler vv
dataset = TwoDimPunchDataset(df, encoded_labels, train_indices=train_indices, max_length=max_length)

In [ ]:
#==============================================
#     3.
#==============================================

from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler


train_sampler = SubsetRandomSampler(train_indices)
val_sampler = SubsetRandomSampler(val_indices)
test_sampler = SubsetRandomSampler(test_indices)


train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, sampler=train_sampler)
val_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, sampler=val_sampler)
test_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, sampler=test_sampler)

In [ ]:
is_cuda = torch.cuda.is_available()

if is_cuda:
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU not available, CPU used")

GPU is available


In [ ]:
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence


def training(model, train_data, n_epochs, lr, criterion, optimizer, patience=10, path="./MyDrive/MyDrive/thesis/models/model1/"):
  counter=0
  val_losses = []
  best_val_loss = 1000
  all_losses = {}
  if type(n_epochs) == type(int()):
    epoch_range = range(1, n_epochs+1)
  else:
    epoch_range = n_epochs

  for epoch in epoch_range:
    model.train()
    optimizer.zero_grad()

    for inputs, labels in train_data:
      inputs = inputs.float()
      #print(f"input size before padding: {inputs.size()}, ")
      #inputs = pad_sequence(inputs)
      #print(f"input size after padding: {inputs.size()}\n")

      inputs, labels = inputs.to(device), labels.to(device)
      model.zero_grad()

      output = model(inputs)
      pred_shape = output.shape

      loss = criterion(output.squeeze(), labels.float())   #squeeze()
      loss.backward()
      optimizer.step()


    model.eval()

    with torch.no_grad():
      for val_inputs, val_labels in val_loader:
        val_inputs = val_inputs.float()
        val_inputs, val_labels = val_inputs.to(device), val_labels.to(device)
        y_pred = model(val_inputs)
        val_loss = criterion(y_pred.squeeze(), val_labels.float())   #squeeze()
        val_losses.append(val_loss)

    mean_val_loss = torch.mean(torch.Tensor(val_losses))
    print("Epoch %d: train loss %.4f, validation loss %.4f" % (epoch, loss, mean_val_loss))
    all_losses[epoch] = (int(loss.cpu()), int(mean_val_loss.cpu()))

    #save checkpoint
    checkpoint_path = os.path.join(path, f"check_epoch_{epoch}.pt")
    torch.save({
            'epoch': epoch,
            #'model_state_dict': model.state_dict(),
            #'optimizer_state_dict': optimizer.state_dict(),
            'train loss': loss,
            'val loss': mean_val_loss
            }, checkpoint_path)


    if mean_val_loss < best_val_loss:
      best_val_loss = mean_val_loss
      counter=0

    elif counter == patience:
      break

    else:
      counter+=1

  #Epoch loop
  #=============================


  try:
    with open(os.path.join(path,'losses.json'), 'w') as f:
      json.dump(all_losses, f)
  except:
    print("Could not save losses.")

  return model

class LSTM_model(nn.Module):
  def __init__(self, input_dim, hidden_dim, num_layers=1, output_dim=4):
    super().__init__()
    self.recurrent = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
    self.dropout = nn.Dropout(0.01)
    self.linear1 = nn.Linear(hidden_dim, hidden_dim//2)
    self.linear2 = nn.Linear(hidden_dim//2, output_dim)
    self.relu = nn.ReLU()
    self.out_layer = nn.Softmax(dim=2)

  def forward(self, input_tensor):
    #input_tensor = pad_sequence(input_tensor)
    x, h_n= self.recurrent(input_tensor)
    x = self.dropout(x)
    x = self.linear1(x)

    x = self.relu(x)
    output = self.linear2(x)
    #print(output)
    output = self.out_layer(output)
    return output

class CONV_model(nn.Module):
  def __init__(self, num_classes=4):
    super().__init__()

    self.conv1 = nn.Conv1d(in_channels = 1, out_channels = 8, kernel_size = 5, padding=3)
    self.maxp1 = nn.MaxPool1d(2)
    self.bn1 = nn.BatchNorm1d(num_features = 8)
    self.relu = nn.ReLU()

    self.conv2 = nn.Conv1d(in_channels = 8, out_channels = 16, kernel_size = 5, padding=3)
    self.maxp2 = nn.MaxPool1d(2)
    self.bn2 = nn.BatchNorm1d(num_features = 16)

    self.conv3 = nn.Conv1d(in_channels = 16, out_channels = 32, kernel_size = 7, padding=4)
    self.maxp3 = nn.MaxPool1d(4)
    self.bn3 = nn.BatchNorm1d(num_features = 32)


    c_ = 47
    self.c_=c_
    self.fc1 = nn.Linear(in_features = 32*c_, out_features = 8*c_)
    self.dropout1 = nn.Dropout(0.01)
    self.fc2 = nn.Linear(in_features = 8*c_, out_features = 1*c_)
    self.out_layer = nn.Linear(in_features = 1*c_, out_features = num_classes)
    self.softmax = nn.Softmax(dim=1)

  def forward(self, input_tensor):
    batch_size = input_tensor.shape[0]

    x = self.conv1(input_tensor)
    x = self.maxp1(x)
    x = self.relu(x)
    x = self.bn1(x)

    x = self.conv2(x)
    x = self.maxp2(x)
    x = self.relu(x)
    x = self.bn2(x)

    x = self.conv3(x)
    x = self.maxp3(x)
    x = self.relu(x)
    x = self.bn3(x)

    x = x.view(batch_size,32*self.c_)

    x = self.dropout1(x)
    x = self.fc1(x)
    x = self.relu(x)
    x = self.dropout1(x)
    x = self.fc2(x)
    x = self.relu(x)
    x = self.dropout1(x)

    output = self.out_layer(x)
    output = self.softmax(output)
    return output




In [ ]:
#==============================
#
# Initialize New Model
#
#==============================


model_name = f"cnn_{period_length}_{sampling_rate}"
#model_name = "temp"
model_path = f"./MyDrive/MyDrive/thesis/models/{model_name}/"
try:
  os.mkdir(model_path)
except:
  print(f"Model path: {model_path} already exists.")

"""
#RNN
if period_length=='raw':
  model = LSTM_model(max_length, 128, num_layers=1, output_dim=4)
else:
  model = LSTM_model(int(n_observations*3), 128, num_layers=1, output_dim=4)
"""
#CNN
if period_length=='raw':
  model = CONV_model(num_classes=4)
else:
  model = CONV_model(num_classes=4)

model.to(device)

n_epochs = 100
learning_rate = 0.0001
decay = 0.0001

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=decay)

patience = 5

Model path: ./MyDrive/MyDrive/thesis/models/cnn_2500_10000/ already exists.


In [ ]:
#==============================
#
# Load Model, resume training
#
#==============================


model_name = f"rnn_{period_length}_{sampling_rate}"
model_path = f"./MyDrive/MyDrive/thesis/models/{model_name}/"

checkpoint = os.listdir(model_path)[-1]
checkpoint_path = os.path.join(model_path, checkpoint)
try:
  os.mkdir(model_path)
except:
  print(f"Model path: {model_path} already exists.")



model = LSTM_model(int(n_observations*3), 256,output_dim=4)
model.load_state_dict(torch.load(checkpoint_path)['model_state_dict'])

model.to(device)

n_epochs = range(int( checkpoint.split('_')[2].split('.')[0] ), 100 + 1)
learning_rate = 0.001

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

patience = 5

Model path: ./MyDrive/MyDrive/thesis/models/rnn_2000_10000/ already exists.


RuntimeError: ignored

In [ ]:
mod1= training(model, train_loader, n_epochs, learning_rate, criterion, optimizer, patience=patience, path=model_path)

Epoch 1: train loss 0.8132, validation loss 0.9006
Epoch 2: train loss 0.7592, validation loss 0.8695
Epoch 3: train loss 1.0276, validation loss 0.8472
Epoch 4: train loss 0.7478, validation loss 0.8343
Epoch 5: train loss 0.7471, validation loss 0.8239
Epoch 6: train loss 0.7455, validation loss 0.8165
Epoch 7: train loss 0.7633, validation loss 0.8117
Epoch 8: train loss 0.7454, validation loss 0.8076
Epoch 9: train loss 0.7548, validation loss 0.8044
Epoch 10: train loss 0.7439, validation loss 0.8010
Epoch 11: train loss 0.7437, validation loss 0.7979
Epoch 12: train loss 0.7658, validation loss 0.7957
Epoch 13: train loss 0.7442, validation loss 0.7935
Epoch 14: train loss 1.0753, validation loss 0.7916
Epoch 15: train loss 0.7439, validation loss 0.7900
Epoch 16: train loss 1.0778, validation loss 0.7885
Epoch 17: train loss 0.7440, validation loss 0.7874
Epoch 18: train loss 0.7437, validation loss 0.7866
Epoch 19: train loss 0.7496, validation loss 0.7855
Epoch 20: train loss 

In [ ]:
torch.save(mod1,f"./MyDrive/MyDrive/thesis/models/{model_name}/{model_name}.pt")

In [ ]:
#===============================
#
# Load best state_dict into model
#
#===============================




model_name = f"rnn_{period_length}_{sampling_rate}"
model_path = f"./MyDrive/MyDrive/thesis/models/{model_name}/"

checkpoint_paths = []
for checkpoint in os.listdir(model_path)[:-1]:
  checkpoint_paths.append(os.path.join(model_path, checkpoint))

best_checkpoint_idx = np.argmin(np.array([ torch.load(checkpoint_path)['loss'] for checkpoint_path in checkpoint_paths]))
best_checkpoint_path = checkpoint_paths[best_checkpoint_idx]
print(best_checkpoint_path)
try:
  os.mkdir(model_path)
except:
  print(f"Model path: {model_path} already exists.")



model = LSTM_model(int(n_observations*3), 256,output_dim=4)
model.load_state_dict(torch.load(best_checkpoint_path)['model_state_dict'])

RuntimeError: ignored

In [ ]:
#!pip install torchmetrics
#===============================
#
# Test model
#
#===============================
from torchmetrics.classification import MulticlassAccuracy
from torchmetrics.classification import F1Score

accuracy_score = MulticlassAccuracy(num_classes=4).to(device)
f1 = F1Score(task='multiclass', num_classes=4, average='macro').to(device)


mod1.eval()
with torch.no_grad():
  count_batches=0
  count_accuracy=0
  count_f1=0
  for inputs, labels in test_loader:
    #inputs = inputs[0,:].float()
    targets = torch.Tensor( [torch.argmax(label) for label in labels] )
    inputs = inputs.float()
    inputs, labels = inputs.to(device), labels.to(device)
    y_pred = mod1(inputs)
    targets = torch.Tensor( [torch.argmax(label) for label in labels] )

    batch_accuracy = accuracy_score(y_pred.squeeze().to(device), targets.to(device))
    batch_f1 = f1(y_pred.squeeze().to(device), targets.to(device))
    count_accuracy+=batch_accuracy
    count_f1+=batch_f1
    count_batches+=1


acc = float((count_accuracy.cpu()/count_batches))
f1 = float((count_f1.cpu()/count_batches))
print(acc, type(acc))
print(f1, type(f1))

columns = ['model', 'period length', 'sampling rate', 'accuracy', 'f1', 'learning rate']
results = pd.DataFrame([['LSTM',period_length, sampling_rate, acc, f1, learning_rate]],columns = columns)
results.to_csv(os.path.join(model_path, f"{model_name}_results.csv"))

0.9666171073913574 <class 'float'>
0.9660329818725586 <class 'float'>
